# Assignment 3

Due March 20th at **10am**.  Pre-grading will be posted on CourseSpaces once the script is running.  Please save your assignment notebook in your `mp248` repo as `mp248/Assignment.3/Assignment.3.ipynb`. Please keep all output and data files in that  `mp248/Assignment.3` notebook.

This assignment leans heavily on material done in Course Notebook 11 and Lab 11.a and 11.b. In the week March 11 -15 we will provide all the usual help in the labs regarding questions concerning Lab 11.a and 11.b and  Course Notebook 11, including discussion possible solutions to the lab problems etc. However, in the lab March 18th we will not be answering any questions concerning those labs or the assignment to treat students in the Monday and Wednesday lab the same. 


## 1 Temperature-dependent network solution

1. Collect the **essential** code components that are required to solve the nuclear network as described in Course Notbook 11, using the `integrate.odeint`, using the rates (for $T9=0.09$, $\rho$ and initial abundances as in the course notebook. Make a plot of the evolution of the mass fractions (`Y*A`) as a function of time in the time interval `[0,1.e6]`s. Make sure all lines have different linestyle and color, as well as a legend. Finally, open a  new file called `results.txt` using `open` (check the docstring for the right `mode` option) and write the mass fraction of $^{15}N$ (`n15`) at $t=10,000s$  and $10^6$ in the first two lines of the file, at the end of a formatted statement which says: `The N15 abundance at t=10000s is: ...` (replace the three dots `...` with the mass fraction value). 

In [ ]:
%pylab ipympl
from scipy import integrate
from scipy import interpolate

In [ ]:
# intial conditions
ini_solar_abu_file = '/home/user/mp248-course-notes/data/iniab1.4E-02As09.ppn'
Z,A,X = loadtxt(ini_solar_abu_file,unpack=True,usecols=(0,2,3))           # select columns with floats and read those
elem = loadtxt(ini_solar_abu_file,unpack=True,usecols=(1),dtype='str')    # select column with character and read that separately. 
X0=array(X,float)
A=array(A,float)
Y0 = X0/A

In [ ]:
global rate

rate=[7.36E-06]       # C12(p,g)
rate.append(3.52E-05) # C13(p,g)
rate.append(2.36E-07) # N14(p,g)
rate.append(2.03E-02) # N15(p,a)

rate = array(rate)
def react_terms(y,rate):
    '''Provide RHS proction terms
    '''
    terms=[]
    terms.append(rate[0]*y[2]*y[0]) # 0 C12(p,g)
    terms.append(rate[1]*y[3]*y[0]) # 1 C13(p,g)
    terms.append(rate[2]*y[4]*y[0]) # 2 N14(p,g)
    terms.append(rate[3]*y[5]*y[0]) # 3 N15(p,a)
    return array(terms)

In [ ]:
def f_rhs(y,t):
    '''Provide RHS for CN network equations''' 

    terms = react_terms(y,rate)

    dh1_dt  =  -terms.sum()
    dhe4_dt =   terms[3]
    dc12_dt =  -terms[0] + terms[3]
    dc13_dt =  -terms[1] + terms[0]
    dn14_dt =  -terms[2] + terms[1]
    dn15_dt =  -terms[3] + terms[2]
    
    return array([dh1_dt,dhe4_dt,dc12_dt,dc13_dt,dn14_dt,dn15_dt])

In [ ]:
# provide list of lines and markers for plotting in loop
from matplotlib import lines
lstyles = list(lines.lineStyles.keys())[0:4]

In [ ]:
dt        = 50.
t         = arange(0,1.e6+0.5*dt,dt)  
markevery = int(100000/dt)

In [ ]:
Y=integrate.odeint(f_rhs,Y0,t)

In [ ]:
close(2);figure(2)
for i in range(len(A)):
    plot(t,log10(Y.T[i]*A[i]),label=elem[i]+str(int(A[i])),fillstyle='none',\
         linestyle=lstyles[mod(i,4)],marker=i+5,markevery=markevery)
legend(loc=4)
ylabel('mass fraction');xlabel('time/s')
#ylim(-2e-6,2e-6)

In [ ]:
fout=open('results.txt',mode='w')
fout.write('The N15 abundance at t=10000s is: %10e \n'%(Y.T[-1][200]*A[-1]))
fout.write('The N15 abundance at t=10**6s is: %10e \n'%(Y.T[-1][-1]*A[-1]))
fout.close()

In [ ]:
cat results.txt

2. Trajectory file 
    - Read file `T-evol.dat` using numpy's `loadtxt` method and combine all data read from the file into one dictionary `traj_data`, so that you can access it like this: `traj_data['T9']` and likewise for key `'time'`. 
    - Plot temperature as a function of time. Use log scale when appropriate. 
    - Using python commands (don't copy and paste!) open again the file `results.txt` (assuming you have closed it previously) and add as the third line the first values of time and temperature contained in the file `T-evol.dat` using the dictionary `traj_data`. Again, check the docstring of the `write` method what `mode=` option is needed to append, and recall that the sting `'\n'` is interpreted as a newline. 

In [ ]:
time,temperature =loadtxt('T-evol.dat',unpack=True)

In [ ]:
time

In [ ]:
close(116),figure(116)
plot(log10(time),temperature,'--o',markevery=5,fillstyle='none',label='temperature/[K]')
legend(loc=0);xlabel('time')

In [ ]:
fout=open('results.txt',mode='a')
fout.write(str(time[0])+' '+str(temperature[0])+'\n')
fout.close()

In [ ]:
str(time[0])+' '+str(temperature[0])

In [ ]:
!cat results.txt

In [ ]:
traj_data = {}
traj_data['T9']  = temperature    # units T9 = 10^9K
traj_data['time'] = time          # units s

In [ ]:
traj_data['time']

In [ ]:
traj_data['T9']

3. Trajectory interpolation: For the integration we need the temperature at any time the solver decides to need. 
    - Use scipy's `interpolate.interp1d` to set up an interpolation function called `T9int`  for `T9` that returns for any time within the limits of the trajectory the  temperature and density, using the linear interpolation option. Make sure your interpolation function has the extrapolation option turned on.
    - Add the output of that function for `t=1.5e+4` and `t=2.9e+7` to the file `results.txt`. 
    

In [ ]:
T9int = interpolate.interp1d(traj_data['time'],traj_data['T9'],\
                 kind='linear',fill_value='extrapolate')

In [ ]:
# write this to results file:
tout=[1.5e+4,2.9e+7]
fout=open('results.txt',mode='a')
fout.write(str(T9int(tout[0]))+' '+str(T9int(tout[1]))+'\n')
fout.close()

In [ ]:
cat results.txt

4. Nuclear data `get_rates` function. 
    - Collect the essential code from Lab 11.a that reads the T-dependent reaction rate data and provides the function `get_rates()` to provide for a given input temperature the `rate` list required in the solution above. Be careful what unit the temperature is in. In the trajectory, as indicated in the header, the unit is plain Kelvin, but in the reaction rate files it is in units of $10^9$K, also referred to as `T9`. 
    - Write the output of `str(get_rates(traj_data['T9'][0]))` as another line to the `results.txt` file. 

In [ ]:
from urllib import request

In [ ]:
# let's use a dictionary to create a simple callable data base structure
nucdata={}
rows = ['T9','adopted','low','high']
files=['12cpg13n.dat','13cpg14n.dat','14npg15o.dat','15npa12c.dat']

In [ ]:
for file in files:
    request.urlretrieve("http://www.astro.ulb.ac.be/nacreii/data/"+file,file)
    nucdata[file]={}
    for i,row in enumerate(rows): nucdata[file][row] = \
        loadtxt(file,unpack=True)[i]

In [ ]:
fints = {}
for file in files:
    fints[file]=interpolate.interp1d(nucdata[file]['T9'],log10(nucdata[file]['adopted']),kind='cubic')

In [ ]:
def get_rates(T, files=files, fints=fints):
    '''Get rates from files for given temperature
    T : float
      temperature in T9, unit 10^9K
      '''
    rate=[]
    for file in files: rate.append(10**(fints[file](T)))
    return rate

In [ ]:
str(get_rates(traj_data['T9'][0]))

In [ ]:
# write to results.txt
fout=open('results.txt',mode='a')
fout.write(str(get_rates(traj_data['T9'][0]))+'\n')
fout.close()

In [ ]:
!cat results.txt

4. Time-dependent temperature network solution. We now have the temperature evolution in function `T9int(time)` and we can get the list `rate` for any temperature. Therefore we can get the rates as a function of time when calculating the RHS. Therefore we will actually use the previously ignored time arguments `t` in `f_rhs(y,t)`. 
    - Identify the code cell from part 1 of this problem that needs to be changed to allow a solution for variable `T9` and therefore variable `rate`. Copy that cell below, and make the required changes. 
    - Use again `odeint` to solve the network ODE, making the necessary change to the function call.
    - Plot all abundances in terms of mass fractions, once more.
    - Add the final abundance of all species in one row as the last line to the `results.txt`.
    - Bonus question: You may find that some abundances are showing erratic behaviour that you may suspect to be the result of lack of accuracy during that particular time of the integration. Study the docstring of the `odeint` method and identify a remedy for this problem, implement it and show the resulting improvement. 

In [ ]:
def f_rhs(y,t):
    '''Provide RHS for CN network equations
    ''' 

    try: 
        T9 = T9int(t)
    except:
        print("Error in f_rhs: %10.3e"%t)
    rate = get_rates(T9)
    terms = react_terms(y,rate)

    dh1_dt  =  -terms.sum()
    dhe4_dt =   terms[3]
    dc12_dt =  -terms[0] + terms[3]
    dc13_dt =  -terms[1] + terms[0]
    dn14_dt =  -terms[2] + terms[1]
    dn15_dt =  -terms[3] + terms[2]
    
    return array([dh1_dt,dhe4_dt,dc12_dt,dc13_dt,dn14_dt,dn15_dt])

In [ ]:
# tester
T9=T9int(t[50])
rate=get_rates(T9)
print(T9)
react_terms(Y0,rate)

In [ ]:
# tester
Y0 = X0/A
f_rhs(Y0,t[1])

In [ ]:
# recall:
# traj_data['time'],traj_data['T9'] are the trajectory data points
# data_int['T9'](time) is the interpolation function that returns T9 for time

# normal solution
#Y=integrate.odeint(f_rhs,Y0,traj_data['time'])

# solution with added attention to a critical period early on in the evolution, soln to bonus question
Y=integrate.odeint(f_rhs,Y0,traj_data['time'],tcrit=traj_data['time'][5:8])

In [ ]:
close(25);figure(25)
for i in range(len(A)):
    plot(log10(traj_data['time'][1:]),log10(Y.T[i][1:]*A[i]),label=elem[i]+str(int(A[i])),fillstyle='none',\
         linestyle=lstyles[mod(i,4)],marker=i+5,markevery=markevery)
legend(loc=4)
ylabel('mass fraction');xlabel('time/s')
#ylim(-2e-6,2e-6)

In [ ]:
# last abundances in mass fraction:
Y[-1]*A

In [ ]:
# write to results.txt
fout=open('results.txt',mode='a')
fout.write(str(Y[-1]*A)+str('\n'))
fout.close()

In [ ]:
! cat results.txt

## 2 Higher-order derivative

In course notebook 5 we introduced a first-order accurate numerical derivative:

### First-order derivative 
The derivative $\frac{df}{dx}$ of a function $y=f(x)$ can be approximated by the difference equation 
$$ f'(x) \approx \frac{f(x+h) -f(x)}{h}.$$

Why? Rearrange the Taylor expansion of $f(x)$
$$
f(x+h) = f(x) + hf^\prime(x) + \frac{1}{2}h^2f^{\prime\prime}(x)
 + \frac{1}{6}h^3f^{\prime\prime\prime}(x) + \dots
$$
to solve for $f^\prime(x)$ and discard order two and higher terms
$$
\frac{1}{2}h^2f^{\prime\prime}(x)
 + \frac{1}{6}h^3f^{\prime\prime\prime}(x) + \dots
$$

### Second-order derivative 
It is very easy to increase the order of the difference equation to second order, and thereby improve the accuracy. The idea is to take into account one more term of the Taylor expansion, then solve again for $f^\prime(x)$ as shown below:

$$
f(x+h) = f(x) + hf^\prime(x) + \frac{1}{2}h^2f^{\prime\prime}(x)
$$
 and solve for $f^\prime(x)$ 
$$
f^\prime(x)  = \frac{f(x+h) - f(x)}{h}  - \frac{1}{2}hf^{\prime\prime}(x) 
$$

with the second order derivative being approximate to first order by
$$
f^{\prime\prime}(x) \approx \frac{f'(x+h) -f'(x)}{h}
$$

* Implement a function `deriv2` which takes the same arguments as `deriv1` introduced in notebook 5, and test it for the third-order polynomial $f(x) = x^3$.
* Create a convergence test plot as in Figure 2 in notebook 5 that shows the dependence of the residual 'log10 (df/dx - 3.0)' as a function of `log10(h)` where $h = 10^{n_{pow}}$ and `npow=[0, -1, ..., -10]`.
* Add to the `results.txt` file the line `Residual first- and second-order for npow=-2: ...` replacing the dots `...` with the two numbers at that value of h for both derivatives.

In [ ]:
deriv1 = lambda f,x,h: (f(x+h) - f(x)) / h
def deriv2(f,x,h):
    deriv1 = lambda f,x,h: (f(x+h) - f(x)) / h
    f2p = (deriv1(f,x+h,h) - deriv1(f,x,h))/h
    fp  = deriv1(f,x,h) - (h/2.)*f2p
    return fp

In [ ]:
f = lambda x: x**3    # function

In [ ]:
deriv2(f,1,1.e-1) -3.

In [ ]:
deriv1(f,1,1.e-1) -3.

In [ ]:
# in order to vectorize wrap the function to isolate the 
# variable over which we should loop turn a scalar function 
# into a vectorized function
# second-order
def hdev(h):
    return deriv2(f,1.,h)
vhdev = vectorize(hdev)

In [ ]:
# first-order

def hdev1(h):
    return deriv1(f,1.,h)
vhdev1 = vectorize(hdev1)

In [ ]:
h_pow = range(0,-11,-1)
h = 10**array(h_pow, dtype=float)

In [ ]:
d2=log10(abs(vhdev(h)-3.0))

In [ ]:
d1=log10(abs(vhdev1(h)-3.0))

In [ ]:
d1

In [ ]:
d2

In [ ]:
close(2);figure(2)
plot(h_pow,d1 ,'o:',label='1st order')
plot(h_pow,d2 ,'*--',label='2nd order')
xlim(0,-10.5), ylabel('log10 (df/dx - 3.0)'), xlabel('log h')
legend()

In [ ]:
# write to results.txt
fout=open('results.txt',mode='a')
fout.write("Residual first- and second-order for npow=-2: %10.4e %10.4e"%(10**d1[2],10**d2[2]))
fout.close()

In [ ]:
cat results.txt

## Notes not part of Assignment solution
### For the record: how I wrote the trajectory
For this problem I am inventing a trajectory that has appropriate time scales for a linearly decreasing temperature from 400MK to 80MK. The time scale for the reactions is proportional to 1/rate. Starting with the highest T and then proceeding to the lowest T means that we start with small time steps which are then increasing, and when plotting the results we can conveniently use a log scale for the x axis. 

In order to create the artificial trajectory below the gunction `get_rates` needs to be defined below first.

In [ ]:
TT=linspace(0.4,0.08)

In [ ]:
thung=0
tt = []
tt.append(thung)
for T in TT[1:]:
    thung = thung + 1./get_rates(T)[2]
    tt.append(thung)

In [ ]:
header="A trajectory with a range of temperature characteristic for \n\
H burning from the hottest conditions found in nova (400MK) to the \n\
coolest conditions found in low-mass main-sequence burning stars. The \n\
time scale for the reactions is scales with 1/rate. Starting with the \n\
highest T and then proceeding to the lowest T means that we start with \n\
small time steps which are then increasing, and when plotting the \n\
results we can conveniently use a log scale for the x axis. \n\
In this way the trajectory demonstrates the network behaviour for \n\
a range of conditions encountered in real stars. \n\
\n\
 Columns: time / [s], T9 / [10**9 K] "
writedata=array([tt,TT])

In [ ]:
savetxt('T-evol.dat',writedata.T,header=header)

In [ ]:
#cat T-evol.dat

In [ ]:

traj_data = {}
traj_data['T9']  = TT
traj_data['time'] = tt
data_int = {}
data_int['T9'] = interpolate.interp1d(traj_data['time'],traj_data['T9'],\
                 kind='linear',fill_value='extrapolate')

In [ ]:
close(2754);figure(2754)
plot(log10(traj_data['time']),traj_data['T9'] )
xlabel('$log_{10} (time/[s])$');ylabel('$T_9 / [10^9K]$')